In [140]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pymongo import MongoClient
from pandas.io.json import json_normalize
from nltk.stem import RSLPStemmer as rsteam
from bs4 import BeautifulSoup
import nltk

%matplotlib inline

In [141]:
questions = pd.read_json('questions.json')
# questions = _questions.join(json_normalize(_questions['info'].tolist()).rename(columns={'test_name':'concurso'})).drop('info',axis=1)

# Pegando os subjects que tem mais questões 

In [142]:
count_questions_subject = questions.groupby('subject').count().reset_index()[['subject','a']].rename(columns={'a':'numero de questoes'})
top_five_subjects = count_questions_subject.sort_values(['numero de questoes'],ascending=False).head(5)['subject'].tolist()
top_five = questions[questions['subject'].isin(top_five_subjects)]

# Visualização 

5 Concursos com mais questões

In [143]:
questions.groupby('subject').count().reset_index()[['subject','a']].rename(columns={'a':'numero de questoes'}).sort_values(['numero de questoes'],ascending=False).head(5)

,subject,numero de questoes
1,Direito Penal,42
3,Semântica,37
2,Direito Processual Civil,34
0,Direito Constitucional,29


# Metodos para pre-processar o texto

In [144]:
def remove_stopwords(document):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    return [word for word in document if word not in stopwords]

def get_text_from_html(document):
    try:
        soup = BeautifulSoup(document,'html.parser')
        text = soup.get_text()
    except:
        text = document
    return text

def steam_list(document):
    steamer = rsteam()
    return [steamer.stem(word) for word in document]

# Sklearn

In [145]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [146]:
top_five['enunciation'] = top_five['enunciation'].apply(
    get_text_from_html).apply(
    nltk.word_tokenize).apply(
    remove_stopwords).apply(
    steam_list)
top_five['enunciation'] = top_five['enunciation'].apply(lambda x: ' '.join(x))

In [147]:
data = top_five[['enunciation','subject']]
X = data['enunciation']
target = np.ravel(data['subject'])

# TD-IDF e Naive Bayes

In [148]:
x_train, x_test, y_train, y_test = train_test_split(X, target,test_size=0.5,random_state=3)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('multiNB', MultinomialNB())
])
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...
        vocabulary=None)), ('multiNB', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [149]:
y = pipeline.predict(x_test)
print(classification_report(y,y_test))

                          precision    recall  f1-score   support

  Direito Constitucional       0.77      1.00      0.87        10
           Direito Penal       1.00      0.53      0.69        34
Direito Processual Civil       0.37      1.00      0.54         7
               Semântica       0.95      1.00      0.98        20

             avg / total       0.89      0.77      0.78        71

